In [1]:
%matplotlib inline
import keras.utils
import os

Using TensorFlow backend.


In [2]:
from __future__ import division, print_function

import os, json
from glob import glob
import numpy as np
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

from keras import backend as K
from keras.layers.normalization import BatchNormalization
from keras.utils.data_utils import get_file
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Convolution2D
from keras.layers import Activation, Dense,GlobalAveragePooling2D

from keras.optimizers import SGD, RMSprop, Adam
from keras.preprocessing import image

#from keras import backend as K
#K.set_image_dim_ordering('tf')
#theno color chaneel come first (3,224,224)

In [3]:
path=r"C:\Users\DE108470\Downloads\images\\"
model_path= path + 'model'
if not os.path.exists(model_path): os.mkdir(model_path)
batch_size=16

In [4]:
def get_batches(dirname, gen=image.ImageDataGenerator(), shuffle=True, batch_size=1, class_mode='categorical',
                target_size=(224,224)):
    return gen.flow_from_directory(dirname, target_size=target_size,
            class_mode=class_mode, shuffle=shuffle, batch_size=batch_size)

In [5]:
train_data=get_batches(path+'train', shuffle=True, batch_size=1,class_mode='categorical',target_size=(224,224))
val_data=get_batches(path+'val', shuffle=True, batch_size=1,class_mode='categorical',target_size=(224,224))

Found 101 images belonging to 2 classes.
Found 34 images belonging to 2 classes.


In [6]:
train_data.classes

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [7]:
total=0
for(rootdir, dirnames, filenames) in os.walk(path):
    total+=len(filenames)
print(total)
valsize=total/10 * 2
valsize

135


27.0

In [8]:
train_class=train_data.classes
train_labels=to_categorical(train_class)
val_class=val_data.classes
val_labels=to_categorical(val_class)
train_labels[:10]

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [9]:
train_labels.shape

(101, 2)

In [10]:
#first dim is nb_samles
sum = np.array([0.0, 0.0, 0.0]);
count = 0
for imgs, labels in train_data:
    #sum += np.sum(np.sum(imgs[], axis=2),axis=1)
    count +=1
    imgs=imgs[0]
    sum += np.sum( np.sum(imgs,axis=1),axis=0)
    if count>= train_data.samples:
        break

avg = sum/(count*224*224)
avg

array([144.86719954, 116.71675425,  96.00903158])

In [11]:
#array([170.14849729, 136.12759088, 110.20172991])

mean pixel value for rgb channel : array([144.86719954, 116.71675425,  96.00903158])


In [12]:
avg= np.array(avg, dtype=np.float32).reshape((1,1,3))
def preprocessing(x):
    x=x-avg
    return x[:, ::-1]


# MLP

In [13]:
def MLP(num_classes): 
    model =  Sequential()
    model.add(Lambda(preprocessing, input_shape=(224,224,3), output_shape=(224,224,3)))
    model.add(Flatten())
    model.add(Dense(4, activation='relu'))
    model.add(Dense(4, activation='relu'))
    model.add(Dense(num_classes,activation='softmax'))
    return model
    
model= MLP(num_classes=2)



In [14]:
opt = keras.optimizers.Adam(lr=0.001)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
   

In [15]:
batch_size=10
epochs=5
history = model.fit_generator(
    train_data,
    validation_data=val_data,
    steps_per_epoch= len(train_data) // batch_size,
    epochs=epochs,
    verbose=1
)


Epoch 1/5
10/10 [==============================] - 2s 207ms/step - loss: 1.6820 - acc: 0.8000 - val_loss: 11.8215 - val_acc: 0.2353
Epoch 2/5
10/10 [==============================] - 2s 152ms/step - loss: 1.7825 - acc: 0.7000 - val_loss: 9.2518 - val_acc: 0.3824
Epoch 3/5
10/10 [==============================] - 2s 159ms/step - loss: 1.8197 - acc: 0.8000 - val_loss: 9.2512 - val_acc: 0.3824
Epoch 4/5
10/10 [==============================] - 2s 156ms/step - loss: 1.7512 - acc: 0.7000 - val_loss: 9.2512 - val_acc: 0.3824
Epoch 5/5
10/10 [==============================] - 2s 150ms/step - loss: 0.0691 - acc: 1.0000 - val_loss: 9.2513 - val_acc: 0.3824


In [17]:
opt2 = keras.optimizers.Adam(lr=0.0001)
model.compile(loss='categorical_crossentropy',
              optimizer=opt2,
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 224, 224, 3)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 150528)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 602116    
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 20        
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 10        
Total params: 602,146
Trainable params: 602,146
Non-trainable params: 0
_________________________________________________________________


In [18]:
batch_size=10
epochs=5
history = model.fit_generator(
    train_data,
    validation_data=val_data,
    steps_per_epoch= len(train_data) // batch_size,
    epochs=epochs,
    verbose=1
)

Epoch 1/5
10/10 [==============================] - 2s 220ms/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 9.2513 - val_acc: 0.3824
Epoch 2/5
10/10 [==============================] - 1s 146ms/step - loss: 0.1387 - acc: 0.9000 - val_loss: 9.2512 - val_acc: 0.3824
Epoch 3/5
10/10 [==============================] - 1s 142ms/step - loss: 3.3617 - acc: 0.8000 - val_loss: 9.2512 - val_acc: 0.3824
Epoch 4/5
10/10 [==============================] - 2s 155ms/step - loss: 0.2071 - acc: 1.0000 - val_loss: 9.2510 - val_acc: 0.3824
Epoch 5/5
10/10 [==============================] - 1s 150ms/step - loss: 4.9044 - acc: 0.7000 - val_loss: 9.2510 - val_acc: 0.3824


loss and acc seems jumps around.
too many parameters at fully connected layers.
val loss is way larger than loss, overfitting.

In [19]:
import numpy as np
def get_data(path, target_size=(224,224)):
    batches = get_batches(path, shuffle=False, batch_size=1, class_mode=None, target_size=target_size)
    return np.concatenate([batches.next() for i in range(batches.samples)])
t_data=get_data(path+'train')
v_data=get_data(path+'val')
t_data.shape

Found 101 images belonging to 2 classes.
Found 34 images belonging to 2 classes.


(101, 224, 224, 3)

# 3convlayer

In [14]:
#no fully connected layer
def ConvNet(num_classes): 
    model =  Sequential()
    model.add(Lambda(preprocessing, input_shape=(224,224,3), output_shape=(224,224,3)))
    
    model.add(Convolution2D(20, kernel_size=(3, 3), strides=(2, 2)))
    model.add(Activation('relu'))
 
    model.add(Convolution2D(40, kernel_size=(3, 3),strides=(2, 2)))
    model.add(Activation('relu'))
    
    model.add(Convolution2D(80, kernel_size=(3, 3),strides=(2, 2)))
    model.add(Activation('relu'))
    
    model.add(GlobalAveragePooling2D())

    model.add(Dense(num_classes,activation='softmax'))
    return model
    
model=ConvNet(num_classes=2)

In [15]:
ConvNet(num_classes=2)

In [16]:
opt = keras.optimizers.Adam(lr=0.0001)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 111, 111, 20)      560       
_________________________________________________________________
activation_1 (Activation)    (None, 111, 111, 20)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 55, 55, 40)        7240      
_________________________________________________________________
activation_2 (Activation)    (None, 55, 55, 40)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 27, 27, 80)        28880     
_________________________________________________________________
activation_3 (Activation)    (None, 27, 27, 80)        0         
__________

In [17]:
batch_size=10
epochs=5
history = model.fit_generator(
    train_data,
    validation_data=val_data,
    steps_per_epoch= len(train_data) // batch_size,
    epochs=epochs,
    verbose=1
)

Epoch 1/5
10/10 [==============================] - 2s 246ms/step - loss: 1.4925 - acc: 0.2000 - val_loss: 0.2170 - val_acc: 0.9412
Epoch 2/5
10/10 [==============================] - 2s 184ms/step - loss: 0.8793 - acc: 0.7000 - val_loss: 0.6727 - val_acc: 0.5588
Epoch 3/5
10/10 [==============================] - 2s 188ms/step - loss: 0.4252 - acc: 0.7000 - val_loss: 0.7776 - val_acc: 0.4412
Epoch 4/5
10/10 [==============================] - 2s 178ms/step - loss: 0.5381 - acc: 0.6000 - val_loss: 0.5377 - val_acc: 0.7941
Epoch 5/5
10/10 [==============================] - 2s 187ms/step - loss: 0.4128 - acc: 0.8000 - val_loss: 0.6415 - val_acc: 0.6176


# More Conv Layer

add padding when feature map w,h is small to take care of corner.


In [13]:
def deepNet(num_classes): 
    model =  Sequential()
    model.add(Lambda(preprocessing, input_shape=(224,224,3), output_shape=(224,224,3)))
    
    model.add(Convolution2D(20, kernel_size=(3, 3), strides=(2, 2)))
    model.add(Activation('relu'))
 
    model.add(Convolution2D(40, kernel_size=(3, 3),strides=(2, 2)))
    model.add(Activation('relu'))
    
    model.add(Convolution2D(80, kernel_size=(3, 3),strides=(2, 2)))
    model.add(Activation('relu'))
    
    model.add(Convolution2D(160, kernel_size=(3, 3),strides=(2, 2),padding='same'))
    model.add(Activation('relu'))
    
    model.add(GlobalAveragePooling2D())

    model.add(Dense(num_classes,activation='softmax'))
    return model
    
model=deepNet(num_classes=2)

In [14]:
opt = keras.optimizers.Adam(lr=0.001)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [15]:
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 111, 111, 20)      560       
_________________________________________________________________
activation_1 (Activation)    (None, 111, 111, 20)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 55, 55, 40)        7240      
_________________________________________________________________
activation_2 (Activation)    (None, 55, 55, 40)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 27, 27, 80)        28880     
_________________________________________________________________
activation_3 (Activation)    (None, 27, 27, 80)        0         
__________

In [16]:
batch_size=10
epochs=5
history = model.fit_generator(
    train_data,
    validation_data=val_data,
    steps_per_epoch= len(train_data) // batch_size,
    epochs=epochs,
    verbose=1
)

Epoch 1/5
10/10 [==============================] - 3s 260ms/step - loss: 8.0623 - acc: 0.5000 - val_loss: 11.3775 - val_acc: 0.2941
Epoch 2/5
10/10 [==============================] - 2s 194ms/step - loss: 9.6709 - acc: 0.4000 - val_loss: 11.3775 - val_acc: 0.2941
Epoch 3/5
10/10 [==============================] - 2s 210ms/step - loss: 1.6118 - acc: 0.9000 - val_loss: 11.3775 - val_acc: 0.2941
Epoch 4/5
10/10 [==============================] - 2s 206ms/step - loss: 6.4472 - acc: 0.6000 - val_loss: 11.3775 - val_acc: 0.2941
Epoch 5/5
10/10 [==============================] - 2s 209ms/step - loss: 6.4472 - acc: 0.6000 - val_loss: 11.3775 - val_acc: 0.2941


Seems not stable

# Batch Norm Layer

In [17]:
def bn():
    model.add(BatchNormalization())

def deepbn(num_classes): 
    model =  Sequential()
    model.add(Lambda(preprocessing, input_shape=(224,224,3), output_shape=(224,224,3)))
    
    model.add(Convolution2D(20, kernel_size=(7, 7), strides=(2, 2)))
    model.add(Activation('relu'))
    bn()
    model.add(Convolution2D(40, kernel_size=(5, 5),strides=(2, 2)))
    model.add(Activation('relu'))
    bn()
    model.add(Convolution2D(80, kernel_size=(3, 3),strides=(2, 2)))
    model.add(Activation('relu'))
    bn()
    model.add(Convolution2D(160, kernel_size=(3, 3),strides=(2, 2),padding='same'))
    model.add(Activation('relu'))
    
    model.add(GlobalAveragePooling2D())

    model.add(Dense(num_classes,activation='softmax'))
    return model
    
model=deepbn(num_classes=2)

In [18]:
opt = keras.optimizers.Adam(lr=0.0001)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [19]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_2 (Lambda)            (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 109, 109, 20)      2960      
_________________________________________________________________
activation_5 (Activation)    (None, 109, 109, 20)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 53, 53, 40)        20040     
_________________________________________________________________
activation_6 (Activation)    (None, 53, 53, 40)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 26, 26, 80)        28880     
_________________________________________________________________
activation_7 (Activation)    (None, 26, 26, 80)        0         
__________

In [20]:
batch_size=10
epochs=10
history = model.fit_generator(
    train_data,
    validation_data=val_data,
    steps_per_epoch= len(train_data) // batch_size,
    epochs=epochs,
    verbose=1
)

Epoch 1/10
10/10 [==============================] - 4s 410ms/step - loss: 1.8895 - acc: 0.5000 - val_loss: 3.5661 - val_acc: 0.2941
Epoch 2/10
10/10 [==============================] - 3s 279ms/step - loss: 0.7760 - acc: 0.5000 - val_loss: 0.3045 - val_acc: 0.9412
Epoch 3/10
10/10 [==============================] - 3s 275ms/step - loss: 0.8240 - acc: 0.7000 - val_loss: 0.4254 - val_acc: 0.7059
Epoch 4/10
10/10 [==============================] - 3s 277ms/step - loss: 1.0421 - acc: 0.6000 - val_loss: 0.8939 - val_acc: 0.3824
Epoch 5/10
10/10 [==============================] - 3s 276ms/step - loss: 0.6749 - acc: 0.8000 - val_loss: 0.7336 - val_acc: 0.5294
Epoch 6/10
10/10 [==============================] - 3s 268ms/step - loss: 0.4769 - acc: 0.8000 - val_loss: 0.9593 - val_acc: 0.4412
Epoch 7/10
10/10 [==============================] - 3s 262ms/step - loss: 0.3786 - acc: 0.8000 - val_loss: 0.4792 - val_acc: 0.6765
Epoch 8/10
10/10 [==============================] - 3s 261ms/step - loss: 0.

# DeepBn

## doule the depth, use stride one, so the final feature maps will be the same size